In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-w7p2/sample_submission.csv
/kaggle/input/2022-ai-w7p2/x_test.csv
/kaggle/input/2022-ai-w7p2/y_train.csv
/kaggle/input/2022-ai-w7p2/x_train.csv


In [2]:
import random
import torch
import torch.nn as nn
import torch.optim as optim

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
device = "cpu"
device

'cpu'

In [4]:
# device = "cuda" if torch.cuda.is_available else "cpu"
# device

# 데이터 전처리

## X_train, y_train, X_test

In [5]:
X_train = pd.DataFrame(pd.read_csv('../input/2022-ai-w7p2/x_train.csv',engine='python',encoding='CP949'))
X_train

,시도명,읍면동명,거주인구,근무인구,방문인구,총 유동인구,평균 속도,평균 소요 시간,평균 기온,일강수량,평균 풍속,월,일,년
0,0,26,76018.965,5009.811,36887.341,117916.117,40.467,21.667,17.750,0.000,2.725,9,25,2019
1,1,24,1232416.968,76043.955,451558.268,1760019.191,38.126,31.632,13.300,14.500,4.500,3,4,2018
2,1,23,134260.946,8412.761,78177.981,220851.688,51.107,45.643,5.250,0.500,4.950,4,7,2018
3,1,25,289327.429,39102.424,287029.303,615459.156,33.067,48.000,26.300,0.500,1.500,8,14,2019
4,1,7,1116617.660,96560.651,524905.301,1738083.612,40.392,26.961,8.400,44.000,4.300,12,1,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21481,0,6,91977.598,14802.819,79885.446,186665.863,41.676,21.676,6.400,0.000,1.600,3,5,2020
21482,0,9,63673.610,3725.663,65813.385,133212.658,48.737,26.105,18.300,7.000,3.700,5,26,2018
21483,1,3,406035.642,46335.550,258079.137,710450.329,49.314,50.943,-6.133,0.833,6.067,1,25,2018
21484,1,25,315737.894,25379.224,283079.548,624196.666,35.312,43.688,8.600,0.000,3.100,3,17,2019


In [6]:
y_train = pd.DataFrame(pd.read_csv('../input/2022-ai-w7p2/y_train.csv',engine='python',encoding='CP949'))
y_train

,교통량
0,0.000
1,1532.989
2,710.107
3,374.200
4,535.882
...,...
21481,91.000
21482,0.000
21483,0.000
21484,283.625


In [7]:
X_test = pd.DataFrame(pd.read_csv('../input/2022-ai-w7p2/x_test.csv',engine='python',encoding='CP949'))
X_test

,시도명,읍면동명,거주인구,근무인구,방문인구,총 유동인구,평균 속도,평균 소요 시간,평균 기온,일강수량,평균 풍속,월,일,년
0,0,28,45737.828,8703.034,91775.025,146215.887,57.286,35.000,6.500,0.0,2.9,1,12,2020
1,1,21,289284.270,48392.749,179649.272,517326.291,50.500,58.912,18.833,0.0,2.3,6,10,2019
2,1,4,72684.218,45632.065,120931.098,239247.380,34.727,60.000,13.200,0.0,1.8,5,10,2019
3,1,12,235752.513,34212.805,186910.679,456875.997,37.167,26.500,11.700,0.0,4.8,3,19,2020
4,1,17,544250.956,64188.293,205182.677,813621.925,31.462,37.769,22.233,0.0,2.3,9,5,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10578,1,37,30949.221,7243.292,73243.733,111436.245,27.500,26.000,0.100,1.0,2.7,2,9,2019
10579,0,39,287544.537,19901.138,230494.219,537939.894,49.130,55.130,20.600,9.0,4.3,7,8,2018
10580,1,35,217597.813,18838.618,150540.187,386976.617,27.500,30.750,0.000,0.0,5.8,2,12,2018
10581,0,6,80350.674,26339.878,101347.505,208038.057,41.000,19.968,18.800,1.0,2.4,10,18,2019


## StandardScaler¶


In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_train

array([[-1.21569523,  0.52199292, -0.8078558 , ...,  0.83551911,
         1.05121316,  0.38288972],
       [ 0.82257459,  0.35319096,  3.06377651, ..., -0.88045638,
        -1.33283577, -1.03576323],
       [ 0.82257459,  0.26878998, -0.61286105, ..., -0.59446047,
        -0.99225735, -1.03576323],
       ...,
       [ 0.82257459, -1.41922962,  0.2970434 , ..., -1.45244821,
         1.05121316, -1.03576323],
       [ 0.82257459,  0.43759194, -0.00527438, ..., -0.88045638,
         0.14300405,  0.38288972],
       [ 0.82257459,  0.6063939 ,  2.59413368, ...,  1.12151502,
        -1.67341418,  0.38288972]])

In [9]:
X_test = scaler.fit_transform(X_test)
X_test

array([[-1.23840082,  0.69153784, -0.91622957, ..., -1.45049013,
        -0.41645411,  1.80493451],
       [ 0.80749301,  0.1072362 , -0.0964397 , ..., -0.02468186,
        -0.64337893,  0.38516181],
       [ 0.80749301, -1.31178206, -0.82552663, ..., -0.30984351,
        -0.64337893,  0.38516181],
       ...,
       [ 0.80749301,  1.27583948, -0.33774001, ..., -1.16532848,
        -0.41645411, -1.03461089],
       [-1.23840082, -1.14483874, -0.79972095, ...,  1.11596477,
         0.26432035,  0.38516181],
       [ 0.80749301, -1.06136708,  2.43939023, ...,  0.54564146,
        -1.66454062,  0.38516181]])

## Tensor & GPU

In [10]:
X_train = torch.FloatTensor(X_train)
X_train

tensor([[-1.2157,  0.5220, -0.8079,  ...,  0.8355,  1.0512,  0.3829],
        [ 0.8226,  0.3532,  3.0638,  ..., -0.8805, -1.3328, -1.0358],
        [ 0.8226,  0.2688, -0.6129,  ..., -0.5945, -0.9923, -1.0358],
        ...,
        [ 0.8226, -1.4192,  0.2970,  ..., -1.4524,  1.0512, -1.0358],
        [ 0.8226,  0.4376, -0.0053,  ..., -0.8805,  0.1430,  0.3829],
        [ 0.8226,  0.6064,  2.5941,  ...,  1.1215, -1.6734,  0.3829]])

In [11]:
y_train = torch.FloatTensor(y_train.to_numpy())
y_train

tensor([[   0.0000],
        [1532.9890],
        [ 710.1070],
        ...,
        [   0.0000],
        [ 283.6250],
        [ 254.1880]])

In [12]:
X_test = torch.FloatTensor(X_test)
X_test

tensor([[-1.2384,  0.6915, -0.9162,  ..., -1.4505, -0.4165,  1.8049],
        [ 0.8075,  0.1072, -0.0964,  ..., -0.0247, -0.6434,  0.3852],
        [ 0.8075, -1.3118, -0.8255,  ..., -0.3098, -0.6434,  0.3852],
        ...,
        [ 0.8075,  1.2758, -0.3377,  ..., -1.1653, -0.4165, -1.0346],
        [-1.2384, -1.1448, -0.7997,  ...,  1.1160,  0.2643,  0.3852],
        [ 0.8075, -1.0614,  2.4394,  ...,  0.5456, -1.6645,  0.3852]])

## Shape

In [13]:
print(X_train.shape)
print(y_train.shape)

torch.Size([21486, 14])
torch.Size([21486, 1])


## DataLoader

In [14]:
mnist_train = torch.utils.data.TensorDataset(X_train, y_train)
mnist_test = torch.utils.data.TensorDataset(X_test)

In [15]:

data_loader_train = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=100,
                                          shuffle=True,
                                          drop_last=True)

data_loader_test = torch.utils.data.DataLoader(dataset=mnist_test,
                                          batch_size=100,
                                          shuffle=True,
                                          drop_last=True)

# Model Training

In [16]:
learning_rate = 0.001
training_epochs = 50
#batch_size = 100
drop_prob = 0.3

In [17]:
linear1 = nn.Linear(14,512)
linear2 = nn.Linear(512,512)
linear3 = nn.Linear(512,512)
linear4 = nn.Linear(512,512)
linear5 = nn.Linear(512,1)
relu = nn.ReLU()
dropout = torch.nn.Dropout(p=drop_prob)

In [18]:
nn.init.xavier_normal_(linear1.weight)
nn.init.xavier_normal_(linear2.weight)
nn.init.xavier_normal_(linear3.weight)
nn.init.xavier_normal_(linear4.weight)
nn.init.xavier_normal_(linear5.weight)

Parameter containing:
tensor([[ 4.7736e-02, -4.0885e-02, -3.8723e-02, -7.8612e-02, -3.4084e-02,
         -1.2083e-01, -5.2914e-03, -4.5715e-03,  8.7628e-02, -8.9595e-04,
          4.6987e-02,  6.8555e-02, -5.8412e-02,  1.0007e-01,  1.1593e-01,
         -1.3659e-01, -6.8932e-02, -5.2407e-02,  8.6386e-03, -7.5976e-03,
          1.5534e-03,  4.1615e-02, -2.3635e-02,  1.2259e-02, -4.5175e-02,
          3.8747e-02, -2.4261e-03,  1.2810e-01, -5.9116e-02,  8.4108e-02,
         -7.0621e-03, -1.1676e-01,  4.3033e-02,  3.1111e-02, -8.7739e-03,
         -3.1920e-02, -4.9295e-03,  1.2657e-02, -5.8699e-02, -4.7530e-02,
          4.7330e-02, -4.9904e-02,  1.4290e-02, -1.6321e-02, -3.1511e-02,
          2.2081e-03, -1.0119e-02,  5.1972e-02, -1.1343e-02, -3.1370e-02,
         -3.2920e-02,  8.3638e-02,  4.5859e-02,  4.5177e-02, -1.8922e-02,
          2.8109e-02, -4.6881e-02,  5.7868e-02,  8.3437e-02,  6.4408e-02,
         -9.8256e-02, -8.7944e-02, -2.3549e-02, -7.9030e-02,  3.9257e-02,
          2.0275

In [19]:
model = nn.Sequential(linear1,relu,dropout,
                     linear2,relu,dropout,
                     linear3,relu,dropout,
                     linear4,relu,dropout,
                     linear5)
model = model.to(device)
model

Sequential(
  (0): Linear(in_features=14, out_features=512, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.3, inplace=False)
  (3): Linear(in_features=512, out_features=512, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.3, inplace=False)
  (6): Linear(in_features=512, out_features=512, bias=True)
  (7): ReLU()
  (8): Dropout(p=0.3, inplace=False)
  (9): Linear(in_features=512, out_features=512, bias=True)
  (10): ReLU()
  (11): Dropout(p=0.3, inplace=False)
  (12): Linear(in_features=512, out_features=1, bias=True)
)

In [20]:
loss = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [21]:
total_batch = len(data_loader_train)
model.train() # 주의사항 drop_out = True

for epoch in range(training_epochs):
    avg_cost = 0
    
    
    for X, Y in data_loader_train:
        X_train = X.to(device)
        y_train = Y.to(device)
        
        optimizer.zero_grad()
        
        hypothesis = model(X_train)
        cost = loss(hypothesis,y_train)
        
        cost.backward()
        optimizer.step()
        
        #평균 error 계산
        avg_cost += cost / total_batch
    
    #if epoch%10 == 0:
    print('Epoch:','%04d' % (epoch+1), 'cost=','{:.9f}'.format(avg_cost))


Epoch: 0001 cost= 166823.234375000
Epoch: 0002 cost= 57887.417968750
Epoch: 0003 cost= 39194.972656250
Epoch: 0004 cost= 30085.683593750
Epoch: 0005 cost= 26185.164062500
Epoch: 0006 cost= 23485.089843750
Epoch: 0007 cost= 21775.996093750
Epoch: 0008 cost= 20287.064453125
Epoch: 0009 cost= 19543.113281250
Epoch: 0010 cost= 17887.886718750
Epoch: 0011 cost= 16736.847656250
Epoch: 0012 cost= 17184.976562500
Epoch: 0013 cost= 15928.783203125
Epoch: 0014 cost= 15817.441406250
Epoch: 0015 cost= 15601.461914062
Epoch: 0016 cost= 14788.504882812
Epoch: 0017 cost= 14816.362304688
Epoch: 0018 cost= 13718.633789062
Epoch: 0019 cost= 13949.270507812
Epoch: 0020 cost= 14117.212890625
Epoch: 0021 cost= 14098.725585938
Epoch: 0022 cost= 13273.527343750
Epoch: 0023 cost= 13403.738281250
Epoch: 0024 cost= 12565.514648438
Epoch: 0025 cost= 13011.339843750
Epoch: 0026 cost= 12325.225585938
Epoch: 0027 cost= 12399.690429688
Epoch: 0028 cost= 12463.539062500
Epoch: 0029 cost= 12275.998046875
Epoch: 0030 c

In [22]:
from torchmetrics import R2Score
r2score = R2Score()

print(r2score(hypothesis,y_train))

tensor(0.9604, grad_fn=<SqueezeBackward0>)


# Testing model

In [23]:
with torch.no_grad():
    model.eval()  # 주의사항 (dropout=False)
    y_test = model(X_test.to(device))
    y_test = np.array(y_test.cpu())
    print(y_test)

[[ 34.570976 ]
 [ -5.598989 ]
 [ -3.3920493]
 ...
 [  2.026814 ]
 [  3.8781066]
 [622.54926  ]]


# Submit

In [24]:
submit = pd.read_csv('../input/2022-ai-w7p2/sample_submission.csv')
submit

,id,predict
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
10578,10578,0
10579,10579,0
10580,10580,0
10581,10581,0


In [25]:
submit['predict']  = y_test
submit

,id,predict
0,0,34.570976
1,1,-5.598989
2,2,-3.392049
3,3,1083.594971
4,4,1753.816895
...,...,...
10578,10578,0.148982
10579,10579,-2.361526
10580,10580,2.026814
10581,10581,3.878107


In [26]:
submit.to_csv('submit.csv',index=False)
